Given a raster file, we want to query and download available USGS 3DEP data. The data should conform to the bounds, the grid size, and projection of the raster file. Options are provided to filter out noise, and generate DTM/DSM, and saving the associated point cloud. We also provide an option to filter out LIDAR returns that exceed a user specified Z-score (default z=3).

In [1]:
# Given a raster file, we want to query and download available USGS 3DEP data. The data should conform to the bounds, the grid size, and projection of the raster file. Options are provided to filter out noise, and generate DTM/DSM, and saving the associated point cloud. We also provide an option to filter out LIDAR returns that exceed a user specified Z-score (default z=3).

# PDAL imports
import pdal

# rasterio imports
import rasterio
from rasterio.warp import transform_bounds
from rasterio import warp
from rasterio.merge import merge

# GIS imports
from pyproj import CRS
from shapely.geometry import Polygon

# Dataframe imports
import pandas as pd
import geopandas as gpd

# Misc imports
import json
import requests
from pathlib import Path

In [ ]:
def return_readers(filename, n_rows = 5, n_cols=5):
    """
    This method takes a raster file and finds overlapping 3DEP data. It then returns a series of readers
    corresponding to non overlapping areas that can be used as part of further PDAL processing pipelines
    """
    with rasterio.open(filename) as ds:
        src_bounds = ds.bounds
        src_crs = ds.crs
        src_transform = ds.transform

    # we want the PDAL pipeline to output DEMs in this CRS
    pointcloud_output_crs = src_crs

    # this assumes the values to be in meters
    x_resolution, y_resolution = abs(src_transform[0]), src_transform[4]
    
    # the point cloud resolution will be determined by the 
    # coarsest resolution available in our raster data
    pointcloud_resolution = max([x_resolution, y_resolution])

    xmin, ymin, xmax, ymax = src_bounds
    x_step = (xmax - xmin) / n_cols
    y_step = (ymax - ymin) / n_rows

    dst_crs = CRS.from_epsg(4326)

    readers = []

    for i in range(int(n_cols)):
        for j in range(int(n_rows)):
            aoi = Polygon.from_bounds(xmin+i*x_step, ymin+j*y_step, xmin+(i+1)*x_step, ymin+(j+1)*y_step)

            src_bounds_transformed = transform_bounds(src_crs, dst_crs, *aoi.bounds)
            aoi_4326 = Polygon.from_bounds(*src_bounds_transformed)

            src_bounds_transformed_3857 = transform_bounds(src_crs, CRS.from_epsg(3857), *aoi.bounds)
            aoi_3857 = Polygon.from_bounds(*src_bounds_transformed_3857)

            gdf = gpd.read_file('https://raw.githubusercontent.com/hobuinc/usgs-lidar/master/boundaries/resources.geojson').set_crs(4326) 
            # in the eventuality that the above URL breaks, we store a local copy
            # gdf = gpd.read_file('../data/shapefiles/resources.geojson').set_crs(4326)

            for _, row in gdf.iterrows():
                if row.geometry.intersects(aoi_4326):
                    usgs_dataset_name = row['name']
                    break

            url = f"https://s3-us-west-2.amazonaws.com/usgs-lidar-public/{usgs_dataset_name}/ept.json"
            reader = {
            "type": "readers.ept",
            "filename": url,
            "resolution": pointcloud_resolution,
            "polygon": str(aoi_3857.wkt),
            }
            
            # SRS associated with the 3DEP dataset
            response = requests.get(url)
            data = response.json()
            srs_wkt = data['srs']['wkt']
            pointcloud_input_crs = CRS.from_wkt(srs_wkt)
            
            readers.append(reader)

    return readers, pointcloud_output_crs, pointcloud_input_crs


In [ ]:
# function that returns a PDAL pipeline to create a pointcloud based on user flags
def create_pdal_pipeline(filter_low_noise=True, filter_high_noise=True, 
                         filter_road=True, reset_classes=False, reclassify_ground=False,
                         return_only_ground=False, percentile_filter=True, z_threshold=3,
                         reproject=True, save_pointcloud=False, 
                         pointcloud_file = 'pointcloud', input_crs=None,
                         output_crs=None, output_type='laz'):
    
    assert output_type in ['las', 'laz'], "Output type must be either 'las' or 'laz'"
    assert output_crs is not None, "Argument 'output_crs' must be explicitly specified!"

    stage_filter_low_noise = {
        "type":"filters.range",
        "limits":"Classification![7:7]"
    }
    stage_filter_high_noise = {
        "type":"filters.range",
        "limits":"Classification![18:18]"
    }
    stage_filter_road = {
        "type":"filters.range",
        "limits":"Classification![11:11]"
    }
    stage_reset_classes = {
        "type":"filters.assign",
        "value":"Classification = 0"
    }
    stage_reclassify_ground = {
        "type":"filters.smrf",
        # added from pdal smrf documentation, in turn from Pingel, 2013
        "scalar":1.2,
        "slope":0.2,
        "threshold":0.45,
        "window":8.0
    }
    stage_percentile_filter =  {
        "type":"filters.python",
        "script":"filter_z.py",
        "pdalargs": {"z_val":z_threshold},
        "function":"filter_z",
        "module":"anything"
    }
    stage_return_ground = {
        "type":"filters.range",
        "limits":"Classification[2:2]"
    }

    stage_reprojection = {
        "type":"filters.reprojection",
        "out_srs":str(output_crs)
    }
    if input_crs is not None:
        stage_reprojection["in_srs"] = str(input_crs)
    
    stage_save_pointcloud_las = {
        "type": "writers.las",
        "filename": f"{pointcloud_file}.las"
    }
    stage_save_pointcloud_laz = {
        "type": "writers.las",
        "compression": "true",
        "minor_version": "2",
        "dataformat_id": "0",
        "filename": f"{pointcloud_file}.laz"
    }

    # Build pipeline
    pipeline = []

    # resetting the original classifications resets 
    # all point classifications to 0 (Unclassified)    
    if reset_classes:
        pipeline.append(stage_reset_classes)
        if reclassify_ground:
            pipeline.append(stage_reclassify_ground)
    else:
        if filter_low_noise:
            pipeline.append(stage_filter_low_noise)
        if filter_high_noise:
            pipeline.append(stage_filter_high_noise)
        if filter_road:
            pipeline.append(stage_filter_road)
        if percentile_filter:
            pipeline.append(stage_percentile_filter)

    # For creating DTMs, we want to process only ground returns
    if return_only_ground:
        pipeline.append(stage_return_ground)
    
    if reproject:
        pipeline.append(stage_reprojection)
    
    # the pipeline can save the pointclouds to a separate file if needed
    if save_pointcloud:
        if output_type == 'laz':
            pipeline.append(stage_save_pointcloud_laz)
        else:
            pipeline.append(stage_save_pointcloud_las)

    return pipeline

In [ ]:
# function that returns a PDAL pipeline to create a DEM based on user flags
def create_dem_stage(dem_filename='dem_output.tif', pointcloud_resolution=1.,
                        gridmethod='idw'):
    dem_stage = {
            "type":"writers.gdal",
            "filename":dem_filename,
            "gdaldriver":'GTiff',
            "nodata":-9999,
            "where":"Z>0", # this ensures that only valid returns in the pointcloud are considered
            "output_type":gridmethod,
            "resolution":float(pointcloud_resolution),
            "gdalopts":"COMPRESS=LZW,TILED=YES,blockxsize=256,blockysize=256,COPY_SRC_OVERVIEWS=YES"
    }
    
    return [dem_stage]

In [ ]:
# raster file for which pointcloud is generated
input_file = '/mnt/working/karthikv/DeepDEM/data/09232024/WV01_20150911_1020010042D39D00_1020010043455300/original_rasters/1020010042D39D00_left_ortho_1.0m_final_aligned_to_REFDEM.tif'

# we use a user specified output srs
crs_file = '/mnt/working/karthikv/DeepDEM/data/09232024/WV01_20150911_1020010042D39D00_1020010043455300/original_rasters/UTM_10N_WGS84_G2139_3D.wkt'
with open(crs_file, 'r') as f:
    OUTPUT_CRS = CRS.from_string(f.read())

# we ignore the CRS returned by this method, since we specify our own
readers, _, INPUT_CRS = return_readers(input_file, n_rows=6, n_cols=6)

# Set pointcloud processing parameters 
FILTER_LOW_NOISE = True
FILTER_HIGH_NOISE = True
FILTER_ROAD = True
RETURN_ONLY_GROUND = False # Set true for DTM
RESET_CLASSES = False
RECLASSIFY_GROUND = False
PERCENTILE_FILTER = True
Z_THRESHOLD = 3 # Z score threshold for filtering noise from "unclassified" class
REPROJECT = True
SAVE_POINTCLOUD=False
POINTCLOUD_RESOLUTION = 1
OUTPUT_TYPE='laz'
GRID_METHOD='idw'

output_path = Path('/mnt/working/karthikv/DeepDEM/data/common_files/tmp')
output_path.mkdir(exist_ok=True)

for i, reader in enumerate(readers):
    dem_file = output_path / f'dem_tile_aoi_{str(i).zfill(4)}.tif'
    pipeline = {'pipeline':[reader]}

    pdal_pipeline = create_pdal_pipeline(
        filter_low_noise=FILTER_LOW_NOISE,
        filter_high_noise=FILTER_HIGH_NOISE,
        filter_road=FILTER_ROAD,
        reset_classes=RESET_CLASSES, reclassify_ground=RECLASSIFY_GROUND,
        return_only_ground=RETURN_ONLY_GROUND, 
        percentile_filter=PERCENTILE_FILTER, z_threshold=Z_THRESHOLD, 
        reproject=REPROJECT,
        save_pointcloud=SAVE_POINTCLOUD, 
        pointcloud_file='pointcloud', input_crs = INPUT_CRS,
        output_crs=OUTPUT_CRS, output_type=OUTPUT_TYPE
    )

    dem_stage = create_dem_stage(dem_filename=str(dem_file), 
                                    pointcloud_resolution=POINTCLOUD_RESOLUTION, 
                                    gridmethod=GRID_METHOD)

    # apply interpolation to fill gaps when generating DTM
    if RETURN_ONLY_GROUND:
        dem_stage[0]['window_size'] = 4
    
    pipeline['pipeline'] += pdal_pipeline
    pipeline['pipeline'] += dem_stage
    pipeline = pdal.Pipeline(json.dumps(pipeline))
    pipeline.execute()

# merge rasters 
files = list(output_path.glob('*.tif'))
merged_raster, transform = merge(files)

with rasterio.open(files[0]) as ds:
    profile_template = ds.profile

profile_template.update({
    'width':merged_raster.shape[-1],
    'height':merged_raster.shape[-2],
    'transform':transform
})
if RETURN_ONLY_GROUND:
    merged_filename = output_path.parent / 'merged_dtm.tif'
else:
    merged_filename = output_path.parent / 'merged_dsm.tif'

with rasterio.open(merged_filename, 'w', **profile_template) as ds:
    ds.write(merged_raster)

# delete temporary files
for file in list(output_path.glob('*.tif')):
    file.unlink()